# Imports 

In [1]:
import tensorflow as tf
#Check GPU and CUDA Compatibility
print(tf.__version__)
print(tf.test.is_built_with_cuda())
print(tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

2.3.0
True
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


In [82]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras import losses
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import glob
import itertools
from collections import defaultdict
import random
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


# Data 

## Dataset Creation 

In [147]:
data_list = glob.glob("./Hand_Gest_Recog/data/*")
data_list

['./Hand_Gest_Recog/data\\00',
 './Hand_Gest_Recog/data\\01',
 './Hand_Gest_Recog/data\\02',
 './Hand_Gest_Recog/data\\03',
 './Hand_Gest_Recog/data\\04',
 './Hand_Gest_Recog/data\\05',
 './Hand_Gest_Recog/data\\06',
 './Hand_Gest_Recog/data\\07',
 './Hand_Gest_Recog/data\\08',
 './Hand_Gest_Recog/data\\09']

In [149]:
for cand in tqdm(data_list):
    d = cand + '/*'
    for gest_type in glob.glob(d):
        d1 = gest_type + '/*'
        imgs = glob.glob(d1)
        images_train = random.sample(imgs, k=160)
        images_test = list(set(imgs) - set(images_train))
        images_valid = random.sample(images_train, k=32)
        images_train = list(set(images_train) - set(images_valid))
        for (i, j, k) in itertools.zip_longest(images_train, images_valid,
                                               images_test):
            s = r'C:/Users/Anuj/Desktop/Work/Data Science/Deep_Learning/Supervised Deep Learning/Part 2 - Convolutional Neural Networks (CNN)/Hand_Gest_Recog/train/' + i[
                26:28] + '/'
            shutil.copy(i, s)
            if (j != None) and (k != None):
                s1 = r'C:/Users/Anuj/Desktop/Work/Data Science/Deep_Learning/Supervised Deep Learning/Part 2 - Convolutional Neural Networks (CNN)/Hand_Gest_Recog/valid/' + j[
                    26:28] + '/'
                s2 = r'C:/Users/Anuj/Desktop/Work/Data Science/Deep_Learning/Supervised Deep Learning/Part 2 - Convolutional Neural Networks (CNN)/Hand_Gest_Recog/test/' + k[
                    26:28] + '/'
                shutil.copy(j, s1)
                shutil.copy(k, s2)
            else:
                continue

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:29<00:00,  2.94s/it]


## Investigating Image Properties 

In [141]:
# load the image
img = load_img(
    './Hand_Gest_Recog/train/01/frame_00_01_0091.png'
    #target_size = (500,500)
)
# report details about the image
print(type(img))
print(img.format)
print(img.mode)
print(img.size)
# show the image
img.show()

<class 'PIL.Image.Image'>
None
RGB
(640, 240)


In [142]:
img_array = img_to_array(img)
print(img_array.shape)
print(img_array.dtype)

(240, 640, 3)
float32


In [66]:
img_array

array([[[ 6.,  6.,  6.],
        [ 5.,  5.,  5.],
        [ 6.,  6.,  6.],
        ...,
        [ 3.,  3.,  3.],
        [ 6.,  6.,  6.],
        [ 3.,  3.,  3.]],

       [[ 6.,  6.,  6.],
        [ 5.,  5.,  5.],
        [ 6.,  6.,  6.],
        ...,
        [ 3.,  3.,  3.],
        [ 6.,  6.,  6.],
        [ 3.,  3.,  3.]],

       [[ 5.,  5.,  5.],
        [ 4.,  4.,  4.],
        [ 6.,  6.,  6.],
        ...,
        [ 2.,  2.,  2.],
        [ 2.,  2.,  2.],
        [ 4.,  4.,  4.]],

       ...,

       [[ 4.,  4.,  4.],
        [ 5.,  5.,  5.],
        [ 4.,  4.,  4.],
        ...,
        [ 4.,  4.,  4.],
        [ 3.,  3.,  3.],
        [ 4.,  4.,  4.]],

       [[ 4.,  4.,  4.],
        [ 5.,  5.,  5.],
        [ 5.,  5.,  5.],
        ...,
        [10., 10., 10.],
        [12., 12., 12.],
        [20., 20., 20.]],

       [[ 4.,  4.,  4.],
        [ 5.,  5.,  5.],
        [ 5.,  5.,  5.],
        ...,
        [10., 10., 10.],
        [12., 12., 12.],
        [20., 20., 20.]]

# Preprocessing

In [156]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('./Hand_Gest_Recog/train/',
                                                 target_size = (200,200),
                                                 batch_size = 32,
                                                 color_mode = 'grayscale',
                                                 class_mode = 'categorical')

Found 12800 images belonging to 10 classes.


In [157]:
valid_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
validating_set = valid_datagen.flow_from_directory('./Hand_Gest_Recog/valid/',
                                                 target_size = (200,200),
                                                 batch_size = 32,
                                                 color_mode = 'grayscale',
                                                 class_mode = 'categorical')

Found 3200 images belonging to 10 classes.


In [158]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('./Hand_Gest_Recog/test/',
                                            target_size = (200,200),
                                            batch_size = 1,
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 3200 images belonging to 10 classes.


# Training 

In [163]:
cnn = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=5, strides=2, activation='relu', input_shape=[200, 200, 1]),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=2, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=2000, activation='relu'),
    tf.keras.layers.Dense(units=1000, activation='relu'),
    tf.keras.layers.Dense(units=500, activation='relu'),
    tf.keras.layers.Dense(units=10, activation='sigmoid')
])

cnn.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 98, 98, 32)        832       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               589952    
_________________________________________________________________
dense_11 (Dense)             (None, 128)              

In [165]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

In [169]:
cnn.fit(x = training_set, validation_data = validating_set, epochs = 15, callbacks = [es])

Epoch 1/15
400/400 [==============================] - 97s 243ms/step - loss: 0.7882 - accuracy: 0.7189 - val_loss: 0.1795 - val_accuracy: 0.9413.1971 - accuracy - ETA: 35s - loss: 1.17 - ETA: 31s - loss: 1.1106 - accuracy: 0 - ETA: 30s - loss: 1.0962 - accuracy: 0.60 - ETA: 30s - loss: - ETA: 24s - loss: 1.0216 - accuracy: 0.634 - ETA: 24s - loss: 1.0 - ETA: 20s - loss: 0.9714 - accuracy: 0 - ETA: 19s - ETA: 2s - loss:
Epoch 2/15
400/400 [==============================] - 85s 214ms/step - loss: 0.1435 - accuracy: 0.9502 - val_loss: 0.0670 - val_accuracy: 0.9791
Epoch 3/15
400/400 [==============================] - 93s 232ms/step - loss: 0.0692 - accuracy: 0.9759 - val_loss: 0.0754 - val_accuracy: 0.9741
Epoch 4/15
400/400 [==============================] - 88s 220ms/step - loss: 0.0536 - accuracy: 0.9826 - val_loss: 0.1009 - val_accuracy: 0.9672
Epoch 5/15
400/400 [==============================] - 87s 218ms/step - loss: 0.0431 - accuracy: 0.9871 - val_loss: 0.2251 - val_accuracy: 0.94

In [171]:
[loss, acc] = cnn.evaluate(test_set)

3200/3200 [==============================] - 16s 5ms/step - loss: 0.1726 - accuracy: 0.9606 0s - loss: 0.1720 - accuracy: 0.


In [172]:
print(acc*100)

96.06249928474426
